In [106]:
import argparse
import requests

from sys import argv
from bs4 import BeautifulSoup, NavigableString, Tag
from math import ceil 
import string

import pandas as pd

import xml.etree.ElementTree as ET

In [2]:
def load_arguments(parser):
    concepto = vars(parser.parse_args(argv[1:]))["c"]
    return concepto

### Scrapear PubMed

In [3]:
def search_pubmed(concepto):
    
    """Para obtener información de la página PubMed a través del uscador de conceptos. Sin embargo el problema es que sólo obtiene resultados en ingles aunque utilicemos el filtro dle idioma"""

    # URL de origen que se 
    url = "https://pubmed.ncbi.nlm.nih.gov/?term={}".format(concepto)
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')

    num_registros = soup.find('div', class_="results-amount").find('span').text.replace(",", "")

    print("Numero de articulos: {}".format(num_registros))

    num_paginas = ceil(int(num_registros)/10)

    #Bucle generando una lista con todos los articulos
    lista_articulos = list()
    for i in range(1,num_paginas+1):
        url_busqueda = url + "&page={}".format(i)
        page = requests.get(url_busqueda)
        soup = BeautifulSoup(page.content, 'html.parser')

        for articulo in soup.find_all('article', class_="full-docsum"):
            lista_articulos.append(articulo.find('a', href=True, class_='docsum-title')['href'])

    #Bucle en el que se mete en cada articulo y obtiene el abstract
    informacion = list()
    for url_articulo in lista_articulos:
        page = requests.get("https://pubmed.ncbi.nlm.nih.gov/" + url_articulo)
        soup = BeautifulSoup(page.content, 'html.parser')

        try:
            informacion.append(soup.find('div', {"id": "enc-abstract"}).text.replace("\n", " "))

        except AttributeError:
            pass

    print("Numero de abstract obtenidos: {} para la palabra: {}".format(len(informacion), conceto))
    return informacion

In [11]:
def search_pub_esp(concepto):
    
    """Para obtener información de una página muy básica que devulve abstracts de PubMed y Medline en español- El problema está al extraer sólo el texto español del abstract ya que no está muy definido el html"""

    # URL de origen que se 
    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) '\
           'AppleWebKit/537.36 (KHTML, like Gecko) '\
           'Chrome/75.0.3770.80 Safari/537.36'}
    
    url = "https://babelmesh.nlm.nih.gov/search/search.php?from=spa&com=&outid=&term={}".format("serologia")
    page = requests.get(url, headers = headers)
    soup = BeautifulSoup(page.content, 'html.parser')

    for br in soup.findAll('br'):
        next_s = br.nextSibling
        if not (next_s and isinstance(next_s,NavigableString)):
            continue
        next2_s = next_s.nextSibling
        if next2_s and isinstance(next2_s,Tag) and next2_s.name == 'hr':
            text = str(next_s).strip().split()[0]

    print("Numero de articulos: {}".format(text))


    num_paginas = ceil(int(text)/20)

    #Bucle generando una lista con todos los articulos
    lista_articulos = list()
    for i in range(1,num_paginas+1):
        url_busqueda = "https://babelmesh.nlm.nih.gov/search/search.php?submit=Y&page={}&Count=3691&QueryKey=1&WebEnv=MCID_5f7dfb66b4eab545651267fb&from=spa&otype=0&com=".format(i)
        page = requests.get(url_busqueda)
        soup = BeautifulSoup(page.content, 'html.parser')

        for articulo in soup.find_all('p'):
#             print(articulo.find('a', href=True)['href'])
            lista_articulos.append(articulo.find('a', href=True)['href'])
            #print(lista_articulos.append(articulo.find('a', href=True, id= "anch_1")['href']))
#             lista_articulos.append(articulo.find('a', href=True, class_='title')['href'])
            
#         for articulo in soup.find_all('li', class_="document source-encyclopedia-spanish"):
#             lista_articulos.append(articulo.find('a', href=True, class_='title')['href'])
            
#         for articulo in soup.find_all('li', class_="document source-medical-sites-spanish"):
#             lista_articulos.append(articulo.find('a', href=True, class_='title')['href'])

    #Bucle en el que se mete en cada articulo y obtiene el abstract
    informacion = list()
    for url_articulo in lista_articulos:
        page = requests.get("https://babelmesh.nlm.nih.gov/search/" + url_articulo)
        soup = BeautifulSoup(page.content, 'html.parser')
        
        for br in soup.findAll('a'):
            next_s = br.nextSibling
            print(next_s)
#             if not (next_s and isinstance(next_s,NavigableString)):
#                 continue
#             next2_s = next_s.nextSibling
#             if next2_s.name == 'Abstract':
#                 text = str(next_s)
#                 print(text)

#         try:
#             informacion.append(soup.find('div', {"class": "mp-content"}).text.replace("\n", " "))

#         except AttributeError:
#             pass

#     print("Numero de abstract obtenidos: {} para la palabra: {}".format(len(informacion), concepto))
#     return informacion

### Scrappear Medline

In [100]:
def search_medline(concepto):
    
    """Para obtener información de la página Medilne a través del uscador de conceptos. A veces cambia la url del articulo"""

    # URL de origen que se 
    url = "https://vsearch.nlm.nih.gov/vivisimo/cgi-bin/query-meta?v%3Aproject=medlineplus-spanish&v%3Asources=medlineplus-spanish-bundle&query={}".format(concepto)
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')

    num_registros = soup.find('span', class_="intronum").text.replace(",", "")

    print("Numero de articulos: {}".format(num_registros))

    num_paginas = ceil(int(num_registros)/10)

    #Bucle generando una lista con todos los articulos
    lista_articulos = list()
    for i in range(0,num_paginas+1):
        url_busqueda = "https://vsearch.nlm.nih.gov/vivisimo/cgi-bin/query-meta?v%3afile=viv_DSsKHJ&server=pvlbsrch14&v:state=root%7Croot-{}-10%7C0&".format(i*10)
        page = requests.get(url_busqueda)
        soup = BeautifulSoup(page.content, 'html.parser')

        for articulo in soup.find_all('li', class_="document source-mp-directory-spanish"):
            lista_articulos.append(articulo.find('a', href=True, class_='title')['href'])
            
        for articulo in soup.find_all('li', class_="document source-encyclopedia-spanish"):
            lista_articulos.append(articulo.find('a', href=True, class_='title')['href'])
            
        for articulo in soup.find_all('li', class_="document source-medical-sites-spanish"):
            lista_articulos.append(articulo.find('a', href=True, class_='title')['href'])

    #Bucle en el que se mete en cada articulo y obtiene el abstract
    informacion = list()
    for url_articulo in lista_articulos:
        page = requests.get("https://vsearch.nlm.nih.gov/vivisimo/cgi-bin/query-meta?v%3Aproject=medlineplus-spanish&v%3Asources=medlineplus-spanish-bundle/" + url_articulo)
        soup = BeautifulSoup(page.content, 'html.parser')
        print(soup)

        try:
            informacion.append(soup.find('div', {"class": "mp-content"}).text.replace("\n", " "))

        except AttributeError:
            pass

    print("Numero de abstract obtenidos: {} para la palabra: {}".format(len(informacion), concepto))
    return informacion

In [101]:
articulos = search_medline("serologia")

Numero de articulos: 26
Numero de abstract obtenidos: 0 para la palabra: serologia


In [96]:
def search_medline_enciclopedia(concepto):
    
    """Para obtener todos los conceptos y su definición que aparecen en la enciclopedia en español de Medline"""

    # URL de origen que se 
    url = "https://medlineplus.gov/spanish/ency/encyclopedia_{}.htm".format(concepto)
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')

    lista_articulos = list()
    for a in soup.find_all('ul', id="index"):
        for articulo in a.find_all('li'):
            lista_articulos.append(articulo.find('a', href=True)['href'])
            
    #Bucle en el que se mete en cada articulo y obtiene el abstract
    informacion = list()
    for url_articulo in lista_articulos:
        page = requests.get("https://medlineplus.gov/spanish/ency/" + url_articulo)
        soup = BeautifulSoup(page.content, 'html.parser')

        try:
            li1 = soup.find('h1', {"class": "with-also"}).text.replace("\n", " ")
            li2 = soup.find('div', {"class": "main-single"}).find('div', {'id':'ency_summary'}).text.replace("\n", " ")
            li3 = soup.find('div', {"class": "main-single"}).find('div',{'class':'section'}).text.replace("\n", " ")
            li = li1 + " "+ li2 + " " + li3
            informacion.append(li)

        except AttributeError:
            pass

    print("Numero de abstract obtenidos: {} para la palabra: {}".format(len(informacion), concepto))
    return informacion

In [98]:
#Obtenemos todos los conceptos para cada letra del abecedario
enciclopedia = []
alpha = list(string.ascii_uppercase)
for letra in alpha:
    enciclopedia.append(search_medline_enciclopedia(letra))

Numero de abstract obtenidos: 330 para la palabra: A
Numero de abstract obtenidos: 86 para la palabra: B
Numero de abstract obtenidos: 350 para la palabra: C
Numero de abstract obtenidos: 217 para la palabra: D
Numero de abstract obtenidos: 457 para la palabra: E
Numero de abstract obtenidos: 82 para la palabra: F
Numero de abstract obtenidos: 67 para la palabra: G
Numero de abstract obtenidos: 146 para la palabra: H
Numero de abstract obtenidos: 260 para la palabra: I
Numero de abstract obtenidos: 2 para la palabra: J
Numero de abstract obtenidos: 2 para la palabra: K
Numero de abstract obtenidos: 96 para la palabra: L
Numero de abstract obtenidos: 151 para la palabra: M
Numero de abstract obtenidos: 79 para la palabra: N
Numero de abstract obtenidos: 54 para la palabra: O
Numero de abstract obtenidos: 293 para la palabra: P
Numero de abstract obtenidos: 21 para la palabra: Q
Numero de abstract obtenidos: 233 para la palabra: R
Numero de abstract obtenidos: 314 para la palabra: S
Nume

In [99]:
#Pasamos la lista a lista planta y lo guardamos en un csv
flat_enciclop = [item for sublist in enciclopedia for item in sublist]
df = pd.DataFrame(flat_enciclop)
df.to_csv('enciclopedia.csv', index=False)

### Obtener XML de MedlineWeb Service

In [124]:
url = "https://wsearch.nlm.nih.gov/ws/query?db=healthTopicsSpanish&term=asma"
page = requests.get(url)

In [127]:
tree = ET.parse(page.text)
# etree.tostring(tree.getroot())
root = tree.tostring().getroot()

FileNotFoundError: [Errno 2] No such file or directory: '<?xml version="1.0" encoding="UTF-8"?>\n<nlmSearchResult>\n  <term>asma</term>\n  <file>viv_GMHBc0</file>\n  <server>pvlbsrch15</server>\n  <count>65</count>\n  <retstart>0</retstart>\n  <retmax>10</retmax>\n  <list num="65" start="0" per="10">\n    <document rank="1" url="https://medlineplus.gov/spanish/asthma.html">\n      <content name="title">&lt;span class="qt0"&gt;Asma&lt;/span&gt;</content>\n      <content name="organizationName">Biblioteca Nacional de Medicina</content>\n      <content name="altTitle">&lt;span class="qt0"&gt;Asma&lt;/span&gt; bronquial</content>\n      <content name="FullSummary">&lt;p&gt;El &lt;span class="qt0"&gt;asma&lt;/span&gt; es una enfermedad crónica que afecta las vías respiratorias. Las vías respiratorias son tubos que llevan el aire hacia dentro y fuera de los pulmones. Si padece de &lt;span class="qt0"&gt;asma&lt;/span&gt;, las paredes internas de sus vías respiratorias se inflaman y se estrec

In [135]:
from urllib.request import urlopen
from xml.etree.ElementTree import parse

In [137]:
var_url = urlopen("https://wsearch.nlm.nih.gov/ws/query?db=healthTopicsSpanish&term=asma")
xmldoc = parse(var_url)

for item in xmldoc.iterfind('nlmSearchResult/list'):
#     title = item.findtext('title')
#     date = item.findtext('pubDate')
#     link = item.findtext('link')

#     print(title)
#     print(date)
#     print(link)
    print(item)

In [130]:
import urllib2
import xmltodict

def homepage(request):
    file = urllib2.urlopen("https://wsearch.nlm.nih.gov/ws/query?db=healthTopicsSpanish&term=asma")
    data = file.read()
    file.close()

    data = xmltodict.parse(data)
    return render_to_response('my_template.html', {'data': data})

ModuleNotFoundError: No module named 'urllib2'

In [ ]:
import requests
import xml.etree.ElementTree as ET

r = requests.get('url',  auth=('user', 'pass'))
tree = ET.parse(r.text)
root = tree.getroot()

In [107]:
# soup = BeautifulSoup(page.content, 'html.parser')

In [120]:
tree = ET.parse('../datasets/medline_asma.xml')
root = tree.getroot()
# print(root.findall('.//Log'))

In [119]:
tree = ET.parse(page.content)
root = tree.getroot()
# print(root.findall('.//Log'))

FileNotFoundError: [Errno 2] No such file or directory: b'<?xml version="1.0" encoding="UTF-8"?>\n<nlmSearchResult>\n  <term>asma</term>\n  <file>viv_0Vby0g</file>\n  <server>pvlbsrch16</server>\n  <count>65</count>\n  <retstart>0</retstart>\n  <retmax>10</retmax>\n  <list num="65" start="0" per="10">\n    <document rank="1" url="https://medlineplus.gov/spanish/asthma.html">\n      <content name="title">&lt;span class="qt0"&gt;Asma&lt;/span&gt;</content>\n      <content name="organizationName">Biblioteca Nacional de Medicina</content>\n      <content name="altTitle">&lt;span class="qt0"&gt;Asma&lt;/span&gt; bronquial</content>\n      <content name="FullSummary">&lt;p&gt;El &lt;span class="qt0"&gt;asma&lt;/span&gt; es una enfermedad cr\xc3\xb3nica que afecta las v\xc3\xadas respiratorias. Las v\xc3\xadas respiratorias son tubos que llevan el aire hacia dentro y fuera de los pulmones. Si padece de &lt;span class="qt0"&gt;asma&lt;/span&gt;, las paredes internas de sus v\xc3\xadas respirat

In [118]:
page.content.getroot()

AttributeError: 'bytes' object has no attribute 'getroot'

In [112]:
for type_tag in soup.findall('list'):
#     value = type_tag.get('foobar')
    print(type_tag)

TypeError: 'NoneType' object is not callable